
# Peer-graded Assignment: Week 3 

## Part 1

Link used for scraping wikipedia: https://towardsdatascience.com/web-scraping-html-tables-with-python-c9baba21059

In [25]:
import pandas as pd # library for data analsysis
import requests 
import lxml.html as lh
import numpy as np

wiki = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

#Create a handle, page, to handle the contents of the website
wiki_handle = requests.get(wiki)

#Store the contents of the website under doc
doc = lh.fromstring(wiki_handle.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    col.append((name[:-1],[]))

#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        
        #Append the data to the empty list of the i'th column
        col[i][1].append(data[:-1])
        #Increment i for the next column
        i+=1

Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

# Dropping all cells with a borough that is Not assigned
df.drop(df.index[df['Borough'] == 'Not assigned'], inplace = True)

print(df.head())

print(df.shape)

print('The dataframe has {} boroughs and {} neighborhoods.'.format(len(df['Borough'].unique()), df.shape[0]))


  Postal code           Borough                                  Neighborhood
2         M3A        North York                                     Parkwoods
3         M4A        North York                              Victoria Village
4         M5A  Downtown Toronto                    Regent Park / Harbourfront
5         M6A        North York             Lawrence Manor / Lawrence Heights
6         M7A  Downtown Toronto  Queen's Park / Ontario Provincial Government
(104, 3)
The dataframe has 11 boroughs and 104 neighborhoods.


## Part 2

In [26]:
geo_data = "http://cocl.us/Geospatial_data"
df_geo = pd.read_csv(geo_data)

df_geo.columns = ['Postal code','Latitude','Longitude']

df_geo.head()

df = pd.merge(df, df_geo, on='Postal code')
df.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494


## Part 3

### Plotting all neighborhoods

In [10]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

# !conda install -c conda-forge --no-deps altair --yes
# !conda install -c conda-forge --no-deps vincent --yes

# !conda install -c conda-forge geopy=1.19.0 --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# !conda install -c conda-forge --no-deps folium=0.5.0 --yes
# !pip install folium
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [11]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="can_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)
# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### Clusering boroughs and plotting with different colours

In [34]:
import random # library for random number generation
import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes

import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 

from sklearn.cluster import KMeans 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Libraries imported.')

Libraries imported.


In [37]:
kclusters = 11
k_means = KMeans(init="k-means++", n_clusters=kclusters, n_init=12)

X = df[['Latitude', 'Longitude']].to_numpy()

k_means.fit(X)
k_means_labels = k_means.labels_
k_means_cluster_centers = k_means.cluster_centers_

df_merged = df
# df_merged.insert(0, 'Cluster Labels', k_means_labels)

print(df.head())

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['Neighborhood'], df_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

   Cluster Labels Postal code           Borough  \
0               2         M3A        North York   
1              10         M4A        North York   
2               0         M5A  Downtown Toronto   
3               3         M6A        North York   
4               0         M7A  Downtown Toronto   

                                   Neighborhood   Latitude  Longitude  
0                                     Parkwoods  43.753259 -79.329656  
1                              Victoria Village  43.725882 -79.315572  
2                    Regent Park / Harbourfront  43.654260 -79.360636  
3             Lawrence Manor / Lawrence Heights  43.718518 -79.464763  
4  Queen's Park / Ontario Provincial Government  43.662301 -79.389494  
